# Baby Names in France – Visualization 3: Gender-Based Name Evolution
This notebook focuses on analyzing the **gender effects** in baby naming trends in France from 1900 to 2020. We particularly explore whether endings of names (the last letter) correlate with gender preferences over time.

## Setup: Install and Import Required Libraries

In [1]:
%pip install altair pandas numpy plotly

Note: you may need to restart the kernel to use updated packages.


"DOSKEY" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
import altair as alt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
# Basic imports
import pandas as pd
import numpy as np
import plotly.graph_objs as go

## Data Loading

In [ ]:
df = pd.read_csv("data/dpt2020.csv", sep=";")

# Check if Corsican departments are present
print("'2A' present?", '2A' in df['dpt'].unique())
print("'2B' present?", '2B' in df['dpt'].unique())

'2A' present? False
'2B' present? False


## Data Cleaning

In [5]:
print(f"Initial shape is {df.shape}")

# Keep only rows where 'annais' and 'dpt' are strictly numeric (or well-formed)
to_clean = ['annais', 'dpt']
for col in to_clean:
    df = df[df[col].str.isnumeric()]

    if col == 'annais':
        df[col] = df[col].astype(int)
    else:
        df[col] = df[col].astype("string")
        df.loc[df['dpt'] == '20', 'dpt'] = '2A'  # Normalize Corsican code
        df = df[~df['dpt'].isin(['971', '972', '973', '974'])]  # Exclude DOM-TOM

    print(df[col].dtype)
    print(f"Shape is now {df.shape}")

Initial shape is (3727553, 5)
int32
Shape is now (3690309, 5)
string
Shape is now (3492154, 5)


## Name Cleaning and Feature Engineering

In [6]:
# Remove rare/aggregate names
df = df[df["preusuel"] != "_PRENOMS_RARES"]
# Add name length
df["longueur"] = df["preusuel"].str.len()
# Remove single-letter names
df = df[df["longueur"] > 1]
print(f"Shape is now {df.shape}")

Shape is now (3471072, 6)


In [7]:
df.head()

,sexe,preusuel,annais,dpt,nombre,longueur
10885,1,AADIL,1983,84,3,5
10886,1,AADIL,1992,92,3,5
10888,1,AAHIL,2016,95,3,5
10892,1,AARON,1962,75,3,5
10893,1,AARON,1976,75,3,5


## Extract Last Letter of Each Name

In [8]:
df['fin'] = df['preusuel'].str[-1].str.lower()

## Prepare Data for Pie Chart

In [9]:
# Aggregate by year and final letter
df_pie = df.groupby(['annais', 'fin'])['nombre'].sum().reset_index()

In [10]:
print(sorted(df['fin'].unique()))

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'â', 'ç', 'é', 'ë', 'ï']


## Function to Aggregate Top 10 Terminations

In [11]:
def top10_terminaisons(df_pie, annee):
    data_annee = df_pie[df_pie['annais'] == annee].copy()
    top5 = data_annee.nlargest(10, 'nombre')
    autres = data_annee[~data_annee['fin'].isin(top5['fin'])]
    autres_sum = autres['nombre'].sum()
    if autres_sum > 0:
        autres_row = pd.DataFrame([{
            'annais': annee,
            'fin': 'autres',
            'nombre': autres_sum
        }])
        top10 = pd.concat([top5, autres_row], ignore_index=True)
    return top10

## Interactive Visualization: Name Terminations Over Time

In [13]:
annee_slider = alt.binding_range(
    min=df_pie['annais'].min(),
    max=df_pie['annais'].max(),
    step=1,
    name="Année :"
)
annee_param = alt.param(
    name="année",
    bind=annee_slider,
    value=df_pie['annais'].min()
)

base = alt.Chart(df_pie).transform_filter(
    alt.datum.annais == annee_param
).transform_window(
    rank='rank(nombre)',
    sort=[alt.SortField('nombre', order='descending')]
).transform_calculate(
    fin_affiche="datum.rank <= 10 ? datum.fin : 'autres'"
).transform_aggregate(
    nombre='sum(nombre)',
    groupby=['fin_affiche']
).transform_window(
    order_rank='rank(-datum.nombre)',
    sort=[alt.SortField('nombre', order='descending')]
).mark_arc().encode(
    theta=alt.Theta("nombre:Q", stack=True),
    color=alt.Color(
        "fin_affiche:N",
        title="Terminaison",
        sort=alt.EncodingSortField(field="nombre", op="sum", order="descending")
    ),
    tooltip=["fin_affiche:N", "nombre:Q"]
).add_params(
    annee_param
).properties(
    title="Répartition des prénoms selon leur terminaison (top 10, autres regroupés)"
)
base

c:\Users\YAEL\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)